# Data processing for logistic regression

## Setup

In [22]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import pickle

In [23]:
# Connect to the s3 bucket
from sagemaker import get_execution_role
import boto3
bucket = 'montrealgr2'
role = get_execution_role()

In [24]:
# Read the datasets from s3 bucket

# dysautonomia
data_key = 'dysautonomia_data.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
dysautonomia_df = pd.read_csv(data_location, low_memory = False)

# lupus
data_key = 'lupus_data.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
lupus_df = pd.read_csv(data_location, low_memory = False)

# Raynaud's
data_key = 'raynauds_data.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
raynauds_df = pd.read_csv(data_location, low_memory = False)

Now we have 3 DataFrames : 
- dysautonomia_df
- lupus_df
- raynauds_df

In [25]:
dysautonomia_df

Unnamed: 0  user_id   age     sex  checkin_date  pressure  \
0             22       36  20.0  female             1       NaN   
1             23       36  20.0  female             2       NaN   
2             24       36  20.0  female             3       NaN   
3             25       36  20.0  female             4       NaN   
4             26       36  20.0  female             5       NaN   
...          ...      ...   ...     ...           ...       ...   
8897       32474    11496  53.0  female            16    1027.0   
8898       32475    11496  53.0  female            17    1017.0   
8899       32476    11496  53.0  female            18    1001.0   
8900       32477    11496  53.0  female            19    1006.0   
8901       32478    11496  53.0  female            20    1010.0   

      temperature_min  temperature_max  exercise_lvl  Dysautonomia  
0                 NaN              NaN           2.0           1.0  
1                 NaN              NaN           NaN           NaN  
2                 NaN              NaN           NaN           1.0  
3                 NaN              NaN           NaN           1.0  
4                 NaN              NaN           NaN           2.0  
...               ...              ...           ...           ...  
8897             32.0             38.0           NaN           2.0  
8898             32.0             42.0           NaN           1.0  
8899             36.0             49.0           NaN           2.0  
8900             32.0             38.0           NaN           2.0  
8901             31.0             34.0           NaN           NaN  

[8902 rows x 10 columns]

## Manage missing values and irrelevant columns

### Remove "Unnamed:0" columns

In [26]:
dysautonomia_df = dysautonomia_df.drop("Unnamed: 0", axis=1)
lupus_df = lupus_df.drop("Unnamed: 0", axis=1)
raynauds_df = raynauds_df.drop("Unnamed: 0", axis=1)

### Deal with NaN values

In [27]:
for col in dysautonomia_df.columns:
    num_na = dysautonomia_df[col].isna().sum()
    print("dysautonomia_df["+str(col)+"]: "+str(num_na)+" NaN values")
print ("\n")

for col in lupus_df.columns:
    num_na = lupus_df[col].isna().sum()
    print("lupus_df["+str(col)+"]: "+str(num_na)+" NaN values")
print("\n")    

for col in raynauds_df.columns:
    num_na = raynauds_df[col].isna().sum()
    print("raynauds_df["+str(col)+"]: "+str(num_na)+" NaN values")
print ("\n")

dysautonomia_df[user_id]: 0 NaN values
dysautonomia_df[age]: 210 NaN values
dysautonomia_df[sex]: 0 NaN values
dysautonomia_df[checkin_date]: 0 NaN values
dysautonomia_df[pressure]: 3848 NaN values
dysautonomia_df[temperature_min]: 3848 NaN values
dysautonomia_df[temperature_max]: 3848 NaN values
dysautonomia_df[exercise_lvl]: 8086 NaN values
dysautonomia_df[Dysautonomia]: 3131 NaN values


lupus_df[user_id]: 0 NaN values
lupus_df[age]: 494 NaN values
lupus_df[sex]: 0 NaN values
lupus_df[checkin_date]: 0 NaN values
lupus_df[humidity]: 3292 NaN values
lupus_df[temperature_min]: 3292 NaN values
lupus_df[temperature_max]: 3292 NaN values
lupus_df[stress_lvl]: 8095 NaN values
lupus_df[Lupus]: 5713 NaN values


raynauds_df[user_id]: 0 NaN values
raynauds_df[age]: 1291 NaN values
raynauds_df[sex]: 0 NaN values
raynauds_df[checkin_date]: 0 NaN values
raynauds_df[humidity]: 3253 NaN values
raynauds_df[pressure]: 3253 NaN values
raynauds_df[temperature_min]: 3253 NaN values
raynauds_df[temperat



What we will do:

Age, humidity, temperature_min, temperature_max, and pressure : fill NaN values with average value

Drop exercise level column because there are too many NaN values

Replace stress level by 0.5 (neutral stress)

Drop rows that have NaN value in the flare up column

In [28]:
# helper function to replace NaN values
def replace_na(df, drop_exercise=True):
    """
    (pandas.DataFrame, bool) -> pandas.DataFrame
    
    Replaces some NaN values of df with appropriate filler values.
    Then drops all the rows that still contain an NaN value.
    """
    
    df = df.copy()
    
    # A dict to store the filler values
    filler_values = {}
    
    # Age
    if "age" in df.columns:
        average_age = round(df.loc[:,"age"].mean())
        filler_values["age"] = average_age
    
    # Humidity
    if "humidity" in df.columns:
        average_humidity = round(df.loc[:,"humidity"].mean())
        filler_values["humidity"] = average_humidity

    # T min
    if "temperature_min" in df.columns:
        average_Tmin = round(df.loc[:,"temperature_min"].mean())
        filler_values["temperature_min"] = average_Tmin
    
    # T max
    if "temperature_max" in df.columns:
        average_Tmax = round(df.loc[:,"temperature_max"].mean())
        filler_values["temperature_max"] = average_Tmax
    
    # pressure
    if "pressure" in df.columns:
        average_pressure = round(df.loc[:,"pressure"].mean())
        filler_values["pressure"] = average_pressure
    
    # stress
    if "stress_lvl" in df.columns:
        filler_values["stress_lvl"] = 0.5
    
    # Drop exercise
    if drop_exercise == True and "exercise_lvl" in df.columns:
        df = df.drop("exercise_lvl", axis=1)
    
    # Replace NaN values as specified above
    df = df.fillna(filler_values)
    
    # Remove rows that still contain NaN values (in the condition flare up column)
    df = df.dropna()
    
    return df

In [29]:
# Modify the DataFrames
dysautonomia_nona_df = replace_na(dysautonomia_df)
lupus_nona_df = replace_na(lupus_df)
raynauds_nona_df = replace_na(raynauds_df)

In [30]:
for col in dysautonomia_nona_df.columns:
    num_na = dysautonomia_nona_df[col].isna().sum()
    print("dysautonomia_df["+str(col)+"]: "+str(num_na)+" NaN values")
print ("\n")

for col in lupus_nona_df.columns:
    num_na = lupus_nona_df[col].isna().sum()
    print("lupus_df["+str(col)+"]: "+str(num_na)+" NaN values")
print("\n")    

for col in raynauds_nona_df.columns:
    num_na = raynauds_nona_df[col].isna().sum()
    print("raynauds_df["+str(col)+"]: "+str(num_na)+" NaN values")
print ("\n")

dysautonomia_df[user_id]: 0 NaN values
dysautonomia_df[age]: 0 NaN values
dysautonomia_df[sex]: 0 NaN values
dysautonomia_df[checkin_date]: 0 NaN values
dysautonomia_df[pressure]: 0 NaN values
dysautonomia_df[temperature_min]: 0 NaN values
dysautonomia_df[temperature_max]: 0 NaN values
dysautonomia_df[Dysautonomia]: 0 NaN values


lupus_df[user_id]: 0 NaN values
lupus_df[age]: 0 NaN values
lupus_df[sex]: 0 NaN values
lupus_df[checkin_date]: 0 NaN values
lupus_df[humidity]: 0 NaN values
lupus_df[temperature_min]: 0 NaN values
lupus_df[temperature_max]: 0 NaN values
lupus_df[stress_lvl]: 0 NaN values
lupus_df[Lupus]: 0 NaN values


raynauds_df[user_id]: 0 NaN values
raynauds_df[age]: 0 NaN values
raynauds_df[sex]: 0 NaN values
raynauds_df[checkin_date]: 0 NaN values
raynauds_df[humidity]: 0 NaN values
raynauds_df[pressure]: 0 NaN values
raynauds_df[temperature_min]: 0 NaN values
raynauds_df[temperature_max]: 0 NaN values
raynauds_df[stress_lvl]: 0 NaN values
raynauds_df[Raynaud's Phenome

### Look at the sex distribution

In [31]:
# Check the distribution of males and females
print("Dysautonomia sexes:", dysautonomia_nona_df["sex"].value_counts(), "\n")
print("Lupus sexes:", lupus_nona_df["sex"].value_counts(), "\n")
print("Raynauds sexes:", raynauds_nona_df["sex"].value_counts())

Dysautonomia sexes: female    5552
male       219
Name: sex, dtype: int64 

Lupus sexes: female    7535
male       134
Name: sex, dtype: int64 

Raynauds sexes: female    4371
male       195
Name: sex, dtype: int64


We choose to only keep females

In [32]:
# Remove males
# dysautonomia
dysautonomia_female_df = dysautonomia_nona_df[dysautonomia_nona_df.loc[:,"sex"]=="female"]
dysautonomia_female_df = dysautonomia_female_df.drop("sex", axis=1)

# lupus
lupus_female_df = lupus_nona_df[lupus_nona_df.loc[:,"sex"]=="female"]
lupus_female_df = lupus_female_df.drop("sex", axis=1)

# Raynauds
raynauds_female_df = raynauds_nona_df[raynauds_nona_df.loc[:,"sex"]=="female"]
raynauds_female_df = raynauds_female_df.drop("sex", axis=1)

### For each disease, create one DataFrame with temperature_min and one DataFrame with temperature_max

In [33]:
# Keep min temp
dysautonomia_minT_df = dysautonomia_female_df.drop("temperature_max", axis=1)
lupus_minT_df = lupus_female_df.drop("temperature_max", axis=1)
raynauds_minT_df = raynauds_female_df.drop("temperature_max", axis=1)

# Keep max temp
dysautonomia_maxT_df = dysautonomia_female_df.drop("temperature_min", axis=1)
lupus_maxT_df = lupus_female_df.drop("temperature_min", axis=1)
raynauds_maxT_df = raynauds_female_df.drop("temperature_min", axis=1)

Now we have the following DataFrames:
- dysautonomia_minT_df
- dysautonomia_maxT_df
- lupus_minT_df
- lupus_maxT_df
- raynauds_minT_df
- raynauds_maxT_df

These DataFrames contain no NaN values, only females, and no exercise column

## Create DataFrames that consider many days in the input

A function that prepares the input array such as to have many days' information in the input

In [34]:
def sliding_window_data (dataframe, window_width):
    """"
    (pandas.DataFrame, int) -> pandas.DataFrame
    
    Converts dataframe into a supervised learning dataset.
    The input in the dataset considers n time steps previous to time t.
    
    dataframe: a dataframe in which the last column is the output value
    window_width: the number of timesteps included in the input
    
    inspired from: https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
    
    >>> test_df = pd.DataFrame()
    >>> test_df["Stress"] = [0, 1, 1, 0]
    >>> test_df["Exercise"] = [1, 4, 3, 1]
    >>> test_df["Flare_up"] = [0, 0, 1, 1]
    >>> print (test_df)
           Stress  Exercise  Flare_up
    0       0         1         0
    1       1         4         0
    2       1         3         1
    3       0         1         1

    
    >>> test_window_df = sliding_window_data(test_df, 3)
    >>> test_window_df
           Stress_(t-3)  Exercise_(t-3)  Flare_up_(t-3)  Stress_(t-2)  Exercise_(t-2)  \
    0           NaN             NaN             NaN           NaN             NaN   
    1           NaN             NaN             NaN           NaN             NaN   
    2           NaN             NaN             NaN           0.0             1.0   
    3           0.0             1.0             0.0           1.0             4.0   

       Flare_up_(t-2)  Stress_(t-1)  Exercise_(t-1)  Flare_up_(t-1)  label  
    0             NaN           NaN             NaN             NaN      0  
    1             NaN           0.0             1.0             0.0      0  
    2             0.0           1.0             4.0             0.0      1  
    3             0.0           1.0             3.0             1.0      1  
    
    """
    
    var_names = list(dataframe.columns)
    n_vars = dataframe.shape[1]
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(window_width, 0, -1):
        cols.append(dataframe.shift(i))
        names += [('%s_(t-%d)' % (var_names[j], i)) for j in range(n_vars)]
    
    # output value
    cols.append(dataframe.iloc[:,-1:])
    names.append("label")
    
    # Put it all together
    window_data = pd.concat(cols, axis=1)
    window_data.columns = names
    return window_data

A function that keeps the long enough sequence of days and arrange them in a sliding window dataset

In [35]:
def prep_dataset_simple(dataframe, window_width):
    """
    (pandas.DataFrame, int) -> pandas.DataFrame
    
    Returns a DataFrame in which we keep only the users who have enough entries to account for the window_width.
    window_width: the number of days past that we want to give as input to the model. 
    The minimum number of entries required = window_width + 1
    In this simple version, it does not matter if the user's entries where uninterrupted or interrupted.
    """
    
    # Iterate through the users in the dataframe to keep the users with enough uninterrupted login days
    users_rows_dict = dict(dataframe["user_id"].value_counts())
    # Keep only the users who have logged in enough days
    good_users = [user for user in users_rows_dict if users_rows_dict[user]>window_width]
    final_window_df = pd.DataFrame()
    # prepare the data according to the window width
    for user in good_users:
        one_usr_df = dataframe[dataframe.loc[:,"user_id"]==user]    
        one_usr_df = one_usr_df.drop(["user_id", 'checkin_date'], axis=1)
        # Put many days in the input
        one_usr_window = sliding_window_data(one_usr_df, window_width)
        # Drop rows with NAN
        one_usr_window = one_usr_window.dropna(axis=0)
        # Append this sequence's rows to all the other sequences' rows
        final_window_df = pd.concat([final_window_df, one_usr_window])
    
    return final_window_df

In [36]:
def prep_dataset_complex(dataframe, window_width):
    """
    (pandas.DataFrame, int) -> pandas.DataFrame
    
    Returns a DataFrame in which we keep only the users who have enough uninterrupted entries to account for the window_width.
    window_width: the number of days past that we want to give as input to the model. 
    The minimum number of entries required = window_width + 1

    """
    
    # Iterate through the users in the dataframe to keep the users with enough uninterrupted login days
    users_rows_dict = dict(dataframe["user_id"].value_counts())
    users_uninter_days_dict = {} # user_id (as key) -> list of list (sequences of uninterrupted login days) (as value)
    for user in users_rows_dict:
        num_rows = users_rows_dict[user]
        login_days = [] # a list to store lists of uninterrupted sequences of days on which a user provided data
        this_usr_df = dataframe[dataframe.loc[:,"user_id"]==user]
        for i, row in this_usr_df.iterrows():
            checkin_date = this_usr_df.loc[i, "checkin_date"]
            # If this date does not immediately follow the previous date, use this data as the start of a new sequence of login days
            if (len(login_days) == 0) or (checkin_date != login_days[-1][-1] + 1):
                new_seq = []
                new_seq.append(checkin_date)
                login_days.append(new_seq)
            # If this date immediately follows the provious checkin_date, add this date to the existing sequence of uninterrupted login days
            else:
                login_days[-1].append(checkin_date)
        # Only keep the sequences of days that are long enough
        good_login_days = []
        for seq in login_days:
            if len(seq) > window_width:
                good_login_days.append(seq)
        # Store the long sequences of days in users_uninter_days_dict
        if len(good_login_days) != 0:
            users_uninter_days_dict[user]=good_login_days
    
    # Keep only the users with long enough login sequences and keep only the days that are part of a long enough login sequence
    # Then create a dataframe in which the input is sequential (many days are included in the input)
    good_window_df = pd.DataFrame()
    
    for user in users_uninter_days_dict:
        one_usr_df = dataframe[dataframe.loc[:,"user_id"]==user]        
        # We will create a dataframe for each day sequence
        for seq in users_uninter_days_dict[user]:
            one_seq_df = one_usr_df[one_usr_df.loc[:,"checkin_date"].isin(seq)]
            one_seq_df = one_seq_df.drop(["user_id", 'checkin_date'], axis=1)
            # Put may days in the input
            one_seq_window = sliding_window_data(one_seq_df, window_width)
            # Drop rows with NAN
            one_seq_window = one_seq_window.dropna(axis=0)
            # Append this sequence's rows to all the other sequences' rows
            good_window_df = pd.concat([good_window_df, one_seq_window])
        
    return good_window_df


### Create the final DataFrames 

For each disease, we will try the following window widths: 1,3,5,10

In [37]:
# A function that prepares the datasets and seves them to a pickle file
def prep_save_data (df, window_width, fname):
    """
    (pd.DataFrame, int, str) -> pd.DataFrame
    
    Returns a new DataFrame created from df and saves this DataFrame to a pickle file called fname. 
    window_width specifies the number of time_steps (days) back that will be considered in the data.
    The new DataFrame contains only one column with age information
    """
    
    new_df = prep_dataset_complex(df, window_width)
    
    # drop duplicate age columns
    all_age_col = []
    all_col = new_df.columns
    for col_name in all_col:
        if "age" in col_name:
            all_age_col.append(col_name)
    extra_age_col = all_age_col.pop(0)
    new_df.drop(extra_age_col, axis=1)
    
    # Save DataFrame to a pickle file
    with open(fname, 'wb') as handle:
        pickle.dump(new_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    return new_df

In [38]:
# save the name of the files to which the dataframes are written
data_fnames = []

min_temp datasets

In [49]:
#fname = "dysautonomia/dysautonomia_minT_w1_df.pickle"
#dysautonomia_minT_w1_df = prep_save_data(dysautonomia_minT_df, 1, fname)
#data_fnames.append(fname)

KeyboardInterrupt: 

In [ ]:
#fname = "dysautonomia/dysautonomia_minT_w3_df.pickle"
#dysautonomia_minT_w3_df = prep_save_data(dysautonomia_minT_df, 3, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "dysautonomia/dysautonomia_minT_w5_df.pickle"
#dysautonomia_minT_w5_df = prep_save_data(dysautonomia_minT_df, 5, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "dysautonomia/dysautonomia_minT_w10_df.pickle"
#dysautonomia_minT_w10_df = prep_save_data(dysautonomia_minT_df, 10, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "lupus/lupus_minT_w1_df.pickle"
#lupus_minT_w1_df = prep_save_data(lupus_minT_df, 1, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "lupus/lupus_minT_w3_df.pickle"
#lupus_minT_w3_df = prep_save_data(lupus_minT_df, 3, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "lupus/lupus_minT_w5_df.pickle"
#lupus_minT_w5_df = prep_save_data(lupus_minT_df, 5, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "lupus/lupus_minT_w10_df.pickle"
#lupus_minT_w10_df = prep_save_data(lupus_minT_df, 10, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "raynauds/raynauds_minT_w1_df.pickle"
#raynauds_minT_w1_df = prep_save_data(raynauds_minT_df, 1, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "raynauds/raynauds_minT_w3_df.pickle"
#raynauds_minT_w3_df = prep_save_data(raynauds_minT_df, 3, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "raynauds/raynauds_minT_w5_df.pickle"
#raynauds_minT_w5_df = prep_save_data(raynauds_minT_df, 5, fname)
#data_fnames.append(fname)

In [ ]:
#fname = "raynauds/raynauds_minT_w10_df.pickle"
#raynauds_minT_w10_df = prep_save_data(raynauds_minT_df, 10, fname)
#data_fnames.append(fname)

max_temp datasets

In [39]:
fname = "oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w1_df.pickle"
dysautonomia_maxT_w1_df = prep_save_data(dysautonomia_maxT_df, 1, fname)
data_fnames.append(fname)

In [40]:
fname = "oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w3_df.pickle"
dysautonomia_maxT_w3_df = prep_save_data(dysautonomia_maxT_df, 3, fname)
data_fnames.append(fname)

In [41]:
fname = "oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w5_df.pickle"
dysautonomia_maxT_w5_df = prep_save_data(dysautonomia_maxT_df, 5, fname)
data_fnames.append(fname)

In [42]:
fname = "oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w10_df.pickle"
dysautonomia_maxT_w10_df = prep_save_data(dysautonomia_maxT_df, 10, fname)
data_fnames.append(fname)

In [43]:
fname = "oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w15_df.pickle"
dysautonomia_maxT_w15_df = prep_save_data(dysautonomia_maxT_df, 15, fname)
data_fnames.append(fname)

In [44]:
fname = "oldgroups/lupus/lupus_oldgroups_maxT_w1_df.pickle"
lupus_maxT_w1_df = prep_save_data(lupus_maxT_df, 1, fname)
data_fnames.append(fname)

In [45]:
fname = "oldgroups/lupus/lupus_oldgroups_maxT_w3_df.pickle"
lupus_maxT_w3_df = prep_save_data(lupus_maxT_df, 3, fname)
data_fnames.append(fname)

In [46]:
fname = "oldgroups/lupus/lupus_oldgroups_maxT_w5_df.pickle"
lupus_maxT_w5_df = prep_save_data(lupus_maxT_df, 5, fname)
data_fnames.append(fname)

In [47]:
fname = "oldgroups/lupus/lupus_oldgroups_maxT_w10_df.pickle"
lupus_maxT_w10_df = prep_save_data(lupus_maxT_df, 10, fname)
data_fnames.append(fname)

In [48]:
fname = "oldgroups/lupus/lupus_oldgroups_maxT_w15_df.pickle"
lupus_maxT_w15_df = prep_save_data(lupus_maxT_df, 15, fname)
data_fnames.append(fname)

In [49]:
fname = "oldgroups/raynauds/raynauds_oldgroups_maxT_w1_df.pickle"
raynauds_maxT_w1_df = prep_save_data(raynauds_maxT_df, 1, fname)
data_fnames.append(fname)

In [50]:
fname = "oldgroups/raynauds/raynauds_oldgroups_maxT_w3_df.pickle"
raynauds_maxT_w3_df = prep_save_data(raynauds_maxT_df, 3, fname)
data_fnames.append(fname)

In [51]:
fname = "oldgroups/raynauds/raynauds_oldgroups_maxT_w5_df.pickle"
raynauds_maxT_w5_df = prep_save_data(raynauds_maxT_df, 5, fname)
data_fnames.append(fname)

In [52]:
fname = "oldgroups/raynauds/raynauds_oldgroups_maxT_w10_df.pickle"
raynauds_maxT_w10_df = prep_save_data(raynauds_maxT_df, 10, fname)
data_fnames.append(fname)

In [53]:
fname = "oldgroups/raynauds/raynauds_oldgroups_maxT_w15_df.pickle"
raynauds_maxT_w15_df = prep_save_data(raynauds_maxT_df, 15, fname)
data_fnames.append(fname)

In [54]:
print (data_fnames)

['oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w1_df.pickle', 'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w3_df.pickle', 'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w5_df.pickle', 'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w10_df.pickle', 'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w15_df.pickle', 'oldgroups/lupus/lupus_oldgroups_maxT_w1_df.pickle', 'oldgroups/lupus/lupus_oldgroups_maxT_w3_df.pickle', 'oldgroups/lupus/lupus_oldgroups_maxT_w5_df.pickle', 'oldgroups/lupus/lupus_oldgroups_maxT_w10_df.pickle', 'oldgroups/lupus/lupus_oldgroups_maxT_w15_df.pickle', 'oldgroups/raynauds/raynauds_oldgroups_maxT_w1_df.pickle', 'oldgroups/raynauds/raynauds_oldgroups_maxT_w3_df.pickle', 'oldgroups/raynauds/raynauds_oldgroups_maxT_w5_df.pickle', 'oldgroups/raynauds/raynauds_oldgroups_maxT_w10_df.pickle', 'oldgroups/raynauds/raynauds_oldgroups_maxT_w15_df.pickle']


# Tests